In [ ]:
from dotenv import load_dotenv

load_dotenv("../timescale-nicocasa.ch-credentials.env")

In [ ]:
import asyncpg
import asyncio

In [ ]:
import os

CONNECTION = os.getenv("TIMESCALE_SERVICE_URL")

In [ ]:
pool = await asyncpg.create_pool(CONNECTION)

In [ ]:
create_table_query = '''
CREATE TABLE card_rank (
    time    TIMESTAMPTZ NOT NULL,
    card_id bpchar      NOT NULL,
    edhrec  INT         NULL
);
'''

async with pool.acquire() as connection:
    # Open a transaction.
    async with connection.transaction():
        # Run the query passing the request argument.
        res = await connection.execute(create_table_query)
        print("Table created", res)

In [ ]:
alter_table_query = '''
    ALTER TABLE card_rank
    ALTER COLUMN edhrec TYPE INT USING edhrec::INTEGER;
'''

async with pool.acquire() as connection:
    # Open a transaction.
    async with connection.transaction():
        # Run the query passing the request argument.
        res = await connection.execute(alter_table_query)
        print("Table updated", res)

In [ ]:
get_columns_query = '''
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'card_rank';
'''

async with pool.acquire() as connection:
    # Run the query to get the columns.
    columns = await connection.fetch(get_columns_query)
    for column in columns:
        print(f"Column: {column['column_name']}, Type: {column['data_type']}")


In [ ]:
set_hypertable = '''
    SELECT create_hypertable(
        'card_rank',
        by_range('time', INTERVAL '7 days')
    );
'''

async with pool.acquire() as connection:
    # Open a transaction.
    async with connection.transaction():
        # Run the query passing the request argument.
        res = await connection.execute(set_hypertable)
        print("Table updated", res)


In [ ]:
import os

DATABASE = os.getenv("DATABASE", "mtg")
DATABASE_HOST = "localhost"
DATABASE_USER = os.getenv("DATABASE_USER", "root")
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD", "root")
DATABASE_PORT = os.getenv("DATABASE_PORT", "27017")

from pymongo import MongoClient

client = MongoClient(
    f"mongodb://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}"
)
db = client[DATABASE]
edhrec_daily = db["edhrec_daily"]

In [ ]:
edhrec_daily.find_one()

In [ ]:
# Fetch all documents from the edhrec_daily collection
documents = edhrec_daily.find()

# Prepare the data for insertion
data_to_insert = (
    (doc['date'], doc['card_id'], doc.get('edhrec', None))
    for doc in documents
)

# Insert data into the card_rank table
insert_query = '''
INSERT INTO card_rank (time, card_id, edhrec)
VALUES ($1, $2, $3)
'''

async with pool.acquire() as connection:
    # async with connection.transaction():
    await connection.executemany(insert_query, data_to_insert)
    print("Data imported successfully")

In [ ]:
count_rows_query = '''
    SELECT COUNT(*) FROM card_rank;
'''

async with pool.acquire() as connection:
    # Run the query to count the rows.
    row_count = await connection.fetchval(count_rows_query)
    print(f"Number of rows in card_rank table: {row_count}")